## AML 3204 Term Project





### Team
* Ankur Kishorbhai Rokad (C0793757)
* Avinash Ravi
* Vishal Sabarinath Venkatesan
* Sahista Patel (C0796681)




### Module 1: Fetching & Sentiment Analysis
*This file has tweet fetching and sentiment analysis process*

In [1]:
import tweepy
import pandas as pd
import json
import numpy as np
import re
import os
from datetime import datetime, date, timedelta
import random
import string
import time

from nltk.corpus import stopwords as st
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import warnings
warnings.filterwarnings('ignore')

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


## Create Tweepy instance

In [2]:
auth = tweepy.OAuthHandler('sspVRoqh0CNUHxpipX5nwAX3H', 'fBlMPlGQGFCVONWSavlIPgAb4XQY9uLWvRZdmSA945NLP35UlE')
auth.set_access_token('866910653495164928-LnEAwUZFU2gZf78o2M0nzEz32yLpma5', 'w4Yx880veGohVYxOHeAp3l2DHF7b1dI0HAMU0hm8J5DUo')

api = tweepy.API(auth)

In [3]:
nltk_stopwords = st.words('english')
stopwords = set(nltk_stopwords)

ps = PorterStemmer()
lm = WordNetLemmatizer()

In [4]:
analyzer = SentimentIntensityAnalyzer()

## Collecting the data

#### Processing the tweets into desired format

In [5]:
def process_tweets(tweets, movie_name, movie_id):
    processed_tweets = []
    for tweet in tweets:       
        text = clean_text(tweet['full_text'])
        tweet['created_at'] = format_date_time(tweet['created_at'])
        processed_tweets.append( {
            'movie_id' :movie_id,
            'movie_name' :movie_name,
            'created_at':tweet['created_at'],
            'tweeter_id': tweet['id_str'],
            'text': text,
            'raw_text' : tweet['full_text'],
            'created_day' : datetime.strftime(datetime.strptime(tweet['created_at'],'%Y-%m-%d %H:%M:%S'),'%Y-%m-%d'),
            'sent_score' : analyzer.polarity_scores(text)
        })
        
    return processed_tweets

def get_json_from_response(response):
    return list(map(lambda a: a._json, response))

#### Generating a data-frame from dictionary

In [6]:
def dict_to_df(tweets):
    df_dict = {
        'movie_id' : list(map(lambda a: a['movie_id'], tweets)),
        'movie_name' : list(map(lambda a: a['movie_name'], tweets)),
        'created_at' : list(map(lambda a: a['created_at'], tweets)),
        'tweeter_id' : list(map(lambda a: a['tweeter_id'], tweets)),
        'text' : list(map(lambda a: a['text'], tweets)),   
        'raw_text' : list(map(lambda a: a['raw_text'], tweets)),     
        'created_day' : list(map(lambda a: a['created_day'], tweets)),
        'sent_score' : list(map(lambda a: a['sent_score'], tweets))
        
    }
    
    df = pd.DataFrame.from_dict(df_dict) 
    return df

#### Cleaning the text

In [7]:
def clean_text(text):

    text = re.sub("http\S+", '', str(text))

    # Contractions ref: https://www.kaggle.com/gunesevitan/nlp-with-disaster-tweets-eda-cleaning-and-bert
    text = re.sub(r"he's", "he is", str(text))
    text = re.sub(r"there's", "there is", str(text))
    text = re.sub(r"We're", "We are", str(text))
    text = re.sub(r"That's", "That is", str(text))
    text = re.sub(r"won't", "will not", str(text))
    text = re.sub(r"they're", "they are", str(text))
    text = re.sub(r"Can't", "Cannot", str(text))
    text = re.sub(r"wasn't", "was not", str(text))
    text = re.sub(r"aren't", "are not", str(text))
    text = re.sub(r"isn't", "is not", str(text))
    text = re.sub(r"What's", "What is", str(text))
    text = re.sub(r"haven't", "have not", str(text))
    text = re.sub(r"hasn't", "has not", str(text))
    text = re.sub(r"There's", "There is", str(text))
    text = re.sub(r"He's", "He is", str(text))
    text = re.sub(r"It's", "It is", str(text))
    text = re.sub(r"You're", "You are", str(text))
    text = re.sub(r"I'M", "I am", str(text))
    text = re.sub(r"shouldn't", "should not", str(text))
    text = re.sub(r"wouldn't", "would not", str(text))
    text = re.sub(r"i'm", "I am", str(text))
    text = re.sub(r"I'm", "I am", str(text))
    text = re.sub(r"Isn't", "is not", str(text))
    text = re.sub(r"Here's", "Here is", str(text))
    text = re.sub(r"you've", "you have", str(text))
    text = re.sub(r"we're", "we are", str(text))
    text = re.sub(r"what's", "what is", str(text))
    text = re.sub(r"couldn't", "could not", str(text))
    text = re.sub(r"we've", "we have", str(text))
    text = re.sub(r"who's", "who is", str(text))
    text = re.sub(r"y'all", "you all", str(text))
    text = re.sub(r"would've", "would have", str(text))
    text = re.sub(r"it'll", "it will", str(text))
    text = re.sub(r"we'll", "we will", str(text))
    text = re.sub(r"We've", "We have", str(text))
    text = re.sub(r"he'll", "he will", str(text))
    text = re.sub(r"Y'all", "You all", str(text))
    text = re.sub(r"Weren't", "Were not", str(text))
    text = re.sub(r"Didn't", "Did not", str(text))
    text = re.sub(r"they'll", "they will", str(text))
    text = re.sub(r"they'd", "they would", str(text))
    text = re.sub(r"DON'T", "DO NOT", str(text))
    text = re.sub(r"they've", "they have", str(text))
    text = re.sub(r"i'd", "I would", str(text))
    text = re.sub(r"should've", "should have", str(text))
    text = re.sub(r"where's", "where is", str(text))
    text = re.sub(r"we'd", "we would", str(text))
    text = re.sub(r"i'll", "I will", str(text))
    text = re.sub(r"weren't", "were not", str(text))
    text = re.sub(r"They're", "They are", str(text))
    text = re.sub(r"let's", "let us", str(text))
    text = re.sub(r"it's", "it is", str(text))
    text = re.sub(r"can't", "cannot", str(text))
    text = re.sub(r"don't", "do not", str(text))
    text = re.sub(r"you're", "you are", str(text))
    text = re.sub(r"i've", "I have", str(text))
    text = re.sub(r"that's", "that is", str(text))
    text = re.sub(r"i'll", "I will", str(text))
    text = re.sub(r"doesn't", "does not", str(text))
    text = re.sub(r"i'd", "I would", str(text))
    text = re.sub(r"didn't", "did not", str(text))
    text = re.sub(r"ain't", "am not", str(text))
    text = re.sub(r"you'll", "you will", str(text))
    text = re.sub(r"I've", "I have", str(text))
    text = re.sub(r"Don't", "do not", str(text))
    text = re.sub(r"I'll", "I will", str(text))
    text = re.sub(r"I'd", "I would", str(text))
    text = re.sub(r"Let's", "Let us", str(text))
    text = re.sub(r"you'd", "You would", str(text))
    text = re.sub(r"It's", "It is", str(text))
    text = re.sub(r"Ain't", "am not", str(text))
    text = re.sub(r"Haven't", "Have not", str(text))
    text = re.sub(r"Could've", "Could have", str(text))
    text = re.sub(r"youve", "you have", str(text))

    # Abbreviations
    text = re.sub("U.S.", "United States", str(text))
    text = re.sub("Dec", "December", str(text))
    text = re.sub("Jan.", "January", str(text))

    # Typos, slang and informal abbreviations
    text = re.sub(r"w/e", "whatever", str(text))
    text = re.sub(r"w/", "with", str(text))
    text = re.sub(r"USAgov", "USA government", str(text))
    text = re.sub(r"recentlu", "recently", str(text))
    text = re.sub(r"Ph0tos", "Photos", str(text))
    text = re.sub(r"amirite", "am I right", str(text))
    text = re.sub(r"exp0sed", "exposed", str(text))
    text = re.sub(r"<3", "love", str(text))
    text = re.sub(r"amageddon", "armageddon", str(text))
    text = re.sub(r"Trfc", "Traffic", str(text))
    text = re.sub(r"WindStorm", "Wind Storm", str(text))
    text = re.sub(r"TRAUMATISED", "traumatized", str(text))

    # Character entity references
    text = re.sub(r"&gt;", ">", str(text))
    text = re.sub(r"&lt;", "<", str(text))
    text = re.sub(r"&amp;", "&", str(text))

    # Remove Urls
    url = re.compile(r'https?://\S+|www\.\S+')
    text = url.sub(r'', text)

    # Remove HTML tags
    html = re.compile(r'<.*?>')
    text = html.sub(r'', text)

    # Remove emoticons, symbols, pictoraphs, transport and map symbols & flags
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Punctuations & special characters
    text = re.sub("[^A-Za-z0-9]+", " ", str(text))

    # Stop word removal
    text = [str(i).lower() for i in text.split() if i.lower() not in stopwords]

    # Stemming the words
    text = [ps.stem(word) for word in text]

    # Lemmatizing the words
    text = [lm.lemmatize(word) for word in text]

    # Joining back to a sentence
    text = ' '.join(text)

    return text

#### Formating the date time

In [8]:
def format_date_time(time_str):
    return datetime.strftime(datetime.strptime(time_str,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')

#### Fetching the tweets

In [9]:
def fetch_tweets(query, count=300):
    return api.search_tweets(q=query,
                             count=count,
                             include_rts=False,
                             tweet_mode='extended')

In [10]:
def generate_file_name(tweeter_id):
    return tweeter_id + "_" + (''.join(
        random.choice(string.ascii_lowercase) for i in range(10)))

## Saving the data

In [11]:
df_movie = pd.read_csv(
    "data/u.item.txt",
    sep="\|",
    names=[
        'item_id', 'title', 'release_date', 'video_release_date', 'IMDb URL',
        'unknown', 'Action', 'Adventure', 'Animation', "Children's", 'Comedy',
        'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
        'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western'
    ])
df_movie.head()

,item_id,title,release_date,video_release_date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
all_tweets = []
for i, movie in df_movie.iterrows():
    print(movie['title'])
    
    # Fetching tweets
    tweets = fetch_tweets(movie['title'])
    
    # Converting raw response to JSON.
    tweets = get_json_from_response(tweets)
    
    # Processing raw JSON, structured.    
    processed_tweets = process_tweets(tweets,movie['title'], movie['item_id'])
    
    all_tweets.extend(processed_tweets)
    time.sleep(45)
df = dict_to_df(all_tweets)  
df.to_csv(f'data/tweets.csv', index=False)
df.head()

Unexpected parameter: include_rts


Toy Story (1995)


In [ ]:
# # Fetching tweets
# tweets = fetch_tweets('Get Shorty (1995)')

# # Converting raw response to JSON.
# tweets = get_json_from_response(tweets)

# # Processing raw JSON, structured.    
# processed_tweets = process_tweets(tweets,'Get Shorty (1995)','1')

# print(processed_tweets)
# df = dict_to_df(processed_tweets)  
# df.to_csv(f'data/twe1ets.csv', index=False)
# df.head()